## Sentiment Analysis Model

In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
import pickle
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

Class for voting which algo has higher accuracy, and confidence in the given answer.

In [2]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers) -> None:
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [3]:
short_pos = open("short_reviews/positive.txt", "r").read()
short_neg = open("short_reviews/negative.txt", "r").read()

documents = []
all_words = []
# J is adjective.
allowed_word_types = ["J"]

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

save_classifier = open("classifiers_pickled/documents.pickle", "wb")
pickle.dump(documents, save_classifier)
save_classifier.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]

save_classifier = open("classifiers_pickled/word_features.pickle", "wb")
pickle.dump(word_features, save_classifier)
save_classifier.close()

Extracting features from words

In [4]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [5]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

save_classifier = open("classifiers_pickled/featuresets.pickle", "wb")
pickle.dump(featuresets, save_classifier)
save_classifier.close()

random.shuffle(featuresets)

### Training Set and Testing Sets

In [25]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:]

OG Naive Bayes classifier

In [26]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("OG Naive Bayes algorithm accuracy percent: ", (nltk.classify.accuracy(classifier, testing_set)) * 100)
classifier.show_most_informative_features(15)

OG Naive Bayes algorithm accuracy percent:  73.19277108433735
Most Informative Features
              engrossing = True              pos : neg    =     21.6 : 1.0
                captures = True              pos : neg    =     17.6 : 1.0
              refreshing = True              pos : neg    =     13.0 : 1.0
                    warm = True              pos : neg    =     13.0 : 1.0
               wonderful = True              pos : neg    =     12.2 : 1.0
               realistic = True              pos : neg    =     11.6 : 1.0
            refreshingly = True              pos : neg    =     11.6 : 1.0
               affecting = True              pos : neg    =     11.0 : 1.0
                  deftly = True              pos : neg    =     11.0 : 1.0
             mesmerizing = True              pos : neg    =     11.0 : 1.0
                provides = True              pos : neg    =     10.6 : 1.0
                    ages = True              pos : neg    =     10.3 : 1.0
            

1. MultinomialNB

In [27]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)

print("MNB_Classifier accuracy percent: ", (nltk.classify.accuracy(MNB_classifier, testing_set)) * 100)

MNB_Classifier accuracy percent:  72.13855421686746


2. BernoulliNB

In [28]:
BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)

print("BNB_classifier accuracy percent: ", (nltk.classify.accuracy(BNB_classifier, testing_set)) * 100)

BNB_classifier accuracy percent:  73.49397590361446


### Classifiers other than Naive Bayes

3. Logistic Regression

In [29]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression(max_iter=10000))
LogisticRegression_classifier.train(training_set)

print("LogisticRegression_Classifier accuracy percent: ", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set)) * 100)

LogisticRegression_Classifier accuracy percent:  73.64457831325302


4. SGDClassifier - Stochastic Gradient Descent learning

In [30]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)

print("SGDClassifier_classifier accuracy percent: ", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set)) * 100)

SGDClassifier_classifier accuracy percent:  72.7409638554217


5. LinearSVC

In [31]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)

print("LinearSVC_classifier accuracy percent: ", (nltk.classify.accuracy(LinearSVC_classifier, testing_set)) * 100)

LinearSVC_classifier accuracy percent:  70.78313253012048


6. NuSVC 

In [32]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)

print("NuSVC_classifier accuracy percent: ", (nltk.classify.accuracy(NuSVC_classifier, testing_set)) * 100)

NuSVC_classifier accuracy percent:  74.3975903614458


Phew! That took a looooong time!

Checks algo and confidence level.

In [33]:
voted_classifier = VoteClassifier(classifier, MNB_classifier, LogisticRegression_classifier, SGDClassifier_classifier, NuSVC_classifier, LinearSVC_classifier)
print("voted_classifier accuracy percent: ", (nltk.classify.accuracy(voted_classifier, testing_set)) * 100)

voted_classifier accuracy percent:  73.19277108433735


Now, let's pickle the whole thing so that we don't have to compute the model again.

In [35]:
import pickle
classifier_list = [classifier, MNB_classifier, BNB_classifier, LogisticRegression_classifier, SGDClassifier_classifier, LinearSVC_classifier, NuSVC_classifier]
save_classifier = open("classifiers_pickled/classifiers.pickle", "wb")
pickle.dump(classifier_list, save_classifier)
save_classifier.close()

Pickling everything in one file was a really bad idea!<br>The file size is 448 MB and GitHub didn't allow me to commit such a huge file.

In [ ]:
def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats)